## Theanets
- [main site](http://theanets.readthedocs.org/en/stable/quickstart.html)
- [git repository](https://github.com/lmjohns3/theanets)

The library implementation and its documents are both elegant and detailed. Please read the online tutorial by [Leif Johnson](http://lmjohns3.com/) if you need to learn the package. I will just try to put some notes here from my learning experience

In [1]:
import theanets

Couldn't import dot_parser, loading of dot files will not be possible.


### main steps in typical workflow
1. Create a Network Layout
2. Train the Network on Data
3. Use the network to make prediction

Typical Usage of the above steps is (http://theanets.readthedocs.org/en/stable/index.html)
```python
import climate
import theanets
import my_data_set

climate.enable_default_logging()

exp = theanets.Experiment(
    theanets.Classifier,
    layers=(100, 200, 100, 10),
)

exp.train(
    my_data_set.training_data,
    my_data_set.validation_data,
    optimize='sgd',
    learning_rate=0.01,
    momentum=0.5,
    hidden_l1=0.1,
)

exp.network.predict(my_data_set.test_data)
```

### Examples of Classifying MNIST

In [10]:
import cPickle
import numpy as np
(train_X, train_y), (valid_X, valid_y), (test_X, test_y) = cPickle.load(open("../data/mnist.pkl"))
train_mnist = (train_X, train_y.astype(np.int32))
valid_mnist = (valid_X, valid_y.astype(np.int32))
test_mnist = (test_X, test_y.astype(np.int32))

In [22]:

## you can stop it at anytime!! 
exp = theanets.Experiment(theanets.Classifier, layers = (28*28, 100, 10))
%time exp.train(train_mnist, valid_mnist, optimize = "sgd", learning_rate = 0.01)

INFO:theanets.layers:layer hid1: 784 -> 100, logistic, 78500 parameters


I 2015-04-28 15:03:59 theanets.layers:377 layer hid1: 784 -> 100, logistic, 78500 parameters


INFO:theanets.layers:layer out: 100 -> 10, softmax, 1010 parameters


I 2015-04-28 15:03:59 theanets.layers:377 layer out: 100 -> 10, softmax, 1010 parameters


INFO:theanets.dataset:valid: 157 of 157 mini-batches of (64, 784) -> (64,)


...

INFO:theanets.trainer:SGD 5 loss=0.279357 err=0.28 hid1<0.1=11.2 hid1<0.9=91.8 out<0.1=86.9 out<0.9=93.1 acc=91.92


I 2015-04-28 15:04:09 theanets.trainer:170 SGD 5 loss=0.279357 err=0.28 hid1<0.1=11.2 hid1<0.9=91.8 out<0.1=86.9 out<0.9=93.1 acc=91.92


INFO:theanets.trainer:interrupted!


I 2015-04-28 15:04:10 theanets.trainer:259 interrupted!
CPU times: user 11 s, sys: 33.6 ms, total: 11.1 s
Wall time: 11.1 s


(OrderedDict([('loss', 0.27935701584927486), ('err', 0.27935701584927486), ('hid1<0.1', 11.19267503196931), ('hid1<0.9', 91.770959878516621), ('out<0.1', 86.940537084398983), ('out<0.9', 93.13598945012788), ('acc', 91.923753196930946)]),
 OrderedDict([('loss', 2.6823372090229118), ('err', 2.6823372090229118), ('hid1<0.1', 3.0099522292993632), ('hid1<0.9', 99.133658439490446), ('out<0.1', 62.269108280254777), ('out<0.9', 100.0), ('acc', 10.051751592356688)]))

In [23]:
model = exp.network
np.mean(np.argmax(model.predict(test_X), axis = 1) == test_y)

0.096299999999999997

In [30]:
model.find??

In [35]:
for i, px in enumerate(model.find(1, 0).get_value().T):
    print i, px.shape

0 (784,)
1 (784,)
2 (784,)
3 (784,)
4 (784,)
5 (784,)
6 (784,)
7 (784,)
8 (784,)
9 (784,)
10 (784,)
11 (784,)
12 (784,)
13 (784,)
14 (784,)
15 (784,)
16 (784,)
17 (784,)
18 (784,)
19 (784,)
20 (784,)
21 (784,)
22 (784,)
23 (784,)
24 (784,)
25 (784,)
26 (784,)
27 (784,)
28 (784,)
29 (784,)
30 (784,)
31 (784,)
32 (784,)
33 (784,)
34 (784,)
35 (784,)
36 (784,)
37 (784,)
38 (784,)
39 (784,)
40 (784,)
41 (784,)
42 (784,)
43 (784,)
44 (784,)
45 (784,)
46 (784,)
47 (784,)
48 (784,)
49 (784,)
50 (784,)
51 (784,)
52 (784,)
53 (784,)
54 (784,)
55 (784,)
56 (784,)
57 (784,)
58 (784,)
59 (784,)
60 (784,)
61 (784,)
62 (784,)
63 (784,)
64 (784,)
65 (784,)
66 (784,)
67 (784,)
68 (784,)
69 (784,)
70 (784,)
71 (784,)
72 (784,)
73 (784,)
74 (784,)
75 (784,)
76 (784,)
77 (784,)
78 (784,)
79 (784,)
80 (784,)
81 (784,)
82 (784,)
83 (784,)
84 (784,)
85 (784,)
86 (784,)
87 (784,)
88 (784,)
89 (784,)
90 (784,)
91 (784,)
92 (784,)
93 (784,)
94 (784,)
95 (784,)
96 (784,)
97 (784,)
98 (784,)
99 (784,)


In [45]:
model.find(1, 1).get_value().shape

(100,)